In [ ]:
from google.colab import drive 
drive.mount('/content/drive')

In [ ]:
!nvidia-smi


In [ ]:
%cd /content/drive/MyDrive/yolov3

Cloning Darknet


In [ ]:
!git clone https://github.com/AlexeyAB/darknet.git

In [ ]:
#install environment from the Makefile
%cd /content/drive/MyDrive/yolov3/darknet
# compute_37, sm_37 for Tesla K80
# compute_75, sm_75 for Tesla T4
# !sed -i 's/ARCH= -gencode arch=compute_60,code=sm_60/ARCH= -gencode arch=compute_75,code=sm_75/g' Makefile

#install environment from the Makefile
#note if you are on Colab Pro this works on a P100 GPU
#if you are on Colab free, you may need to change the Makefile for the K80 GPU
#this goes for any GPU, you need to change the Makefile to inform darknet which GPU you are running on.
!sed -i 's/OPENCV=0/OPENCV=1/g' Makefile
!sed -i 's/GPU=0/GPU=1/g' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/g' Makefile
!sed -i "s/ARCH= -gencode arch=compute_60,code=sm_60/ARCH= ${ARCH_VALUE}/g" Makefile
!make

In [ ]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [ ]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg

In [ ]:
!mkdir "/content/drive/MyDrive/yolov3/backup"

In [ ]:
!echo "no_mask" > data/obj/obj.names
!echo "mask" > data/obj/obj.names
!echo -e 'classes= 1\ntrain  = /content/drive/MyDrive/yolov3/data/obj/train.txt\nvalid  = /content/drive/MyDrive/yolov3/data/obj/test.txt\nnames = /content/drive/MyDrive/yolov3/data/obj/obj.names\nbackup = /content/drive/MyDrive/yolov3/backup' > /content/drive/MyDrive/yolov3/data/obj/obj.data

In [ ]:
!wget https://pjreddie.com/media/files/darknet53.conv.74

In [ ]:
import glob
images_list = glob.glob("/content/drive/MyDrive/yolov3/data/obj/Dataset/*.jpg")
print(images_list)

In [ ]:
#Create training.txt file
file = open("/content/drive/MyDrive/yolov3/data/obj/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 

In [ ]:
data_file = '/content/drive/MyDrive/yolov3/data/obj/obj.data'
cfg_file = '/content/drive/MyDrive/yolov3/darknet/cfg/yolov3_training.cfg'
# for initial
weights = '/content/drive/MyDrive/yolov3/darknet/darknet53.conv.74'
# for retraining with final weights to more iterations
weights = '/content/drive/MyDrive/yolov3/backup'

In [ ]:
%cd ..
!chmod +x darknet/darknet


In [ ]:
%cd /content/drive/MyDrive/yolov3/darknet
# Start the training
!./darknet detector train {data_file} {cfg_file} {weights} -dont_show -map